## Image cleaning with **OPEN CV**

En este notebook he puesto en práctica la herramienta de *computer vision* adaptada para python OPEN-CV. En las siguientes celdas he desarrollado el proceso de encontrar una función capaz de aislar los objetos de las imágenes de forma que sea posible contarlas con un algoritmo de inteligencia artificial

---

**DEPENDENCIAS**

Importamos las dependencias y librerías necesarias para el desarrollo

In [53]:
import os
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

**RUTAS**

Creo la ruta para las imágenes de entrenamiento y prueba y una función para cargar al modelo con todas las imágenes de la dirección dada

In [54]:
train_csv_path = './data/train.csv'
test_csv_path = './data/test.csv'
sol_csv_path = './data/sample_submision.csv'

TRAIN_PATH = './train/train/'
TEST_PATH = './test/test/'
path_sol = './test_2/solution/'

**PRUEBAS**

Leo una única imagen para realizar las pruebas de limpieza, efectos y filtrado

In [55]:
img = cv.imread('./train/train/clips-30002.png')

In [56]:
img.shape

(256, 256, 3)

In [57]:
cv.imshow('img', img)

cv.waitKey(0)
cv.destroyAllWindows()

Mi primera forma de afrontar el reto es dividiendo la imagen original en los **tres** canales de color que componen cualquier imagen en escala *RGB*. La imagen consta de unos cuatro colores en total: gris, blanco, rojo y azul. 

Para eliminar las líneas que ensucian mi imagen y meten ruido a mi detección de objetos, me quedo con la imagen en uno de esos canales únicamente para así eliminarme las líneas del mismo color predominante, el azul

**LIMPIEZA Y LÍNEAS HORIZONTALES**

In [58]:
#transformo la imagen a rgb, que por defecto es interpretada por opencv como BGR
rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

In [59]:
#espacio en blanco para la máscara
blank = np.zeros(img.shape[:2], dtype='uint8')

#divido por bandas de color
b,g,r = cv.split(img)

#uno con el espacio vacío cada color
blue = cv.merge([b,blank,blank])
green = cv.merge([blank,g,blank])
red = cv.merge([blank,blank,r])

#la azul es la que mejor funciona

cv.imshow('blue', b)
cv.waitKey(0)
cv.destroyAllWindows()

In [60]:
#aplico un desenfoque para que el modelo encuentre a grosso modo los objetos
blur = cv.GaussianBlur(b, (3,3), cv.BORDER_DEFAULT)

In [61]:
#binarizamos la imagen a partir del desenfoque de la capa de azul
ret, thresh = cv.threshold(blur, 225, 255, 1, cv.THRESH_BINARY)

In [62]:
#la invertimos
thresh = 255 - thresh

In [63]:
#destaco los contornos y hago pruebas de valores
canny = cv.Canny(thresh, 0, 25)

In [64]:
#dilatamos los contornos obtenidos
dilated = cv.dilate(canny, (15,15), iterations=2)

In [65]:
#erosionamos la dilatación previa
eroded = cv.erode(dilated, (7,7), iterations=1)
cv.imshow('dilated', dilated)
cv.imwrite('./img/imagen_limpia.jpg', eroded)
cv.waitKey(0)

-1

![Imagen limpia hasta ahora](\img\imagen_limpia.jpg)

Aquí podemos ver que el trabajo hasta ahora de limpieza ha resultado efectivo y que hemos conseguido aislar casi del todo los clips del resto del fondo. A falta de la línea vertical de la izquierda que marca el margen del cuaderno. Para esa línea, aplicaremos una máscara que detecte líneas verticales en la imagen y sustituya los píxeles

In [66]:
contours, hierarchy = cv.findContours(eroded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
print(f'{len(contours)} contour(s) found!')

3 contour(s) found!


Si hacemos un conteo de contornos empleando la propia herramienta de *OPEN CV*, podemos ver que los contornos encontrados coinciden con los contornos de la imagen (dos clips y la línea)

**LÍNEAS VERTICALES**

In [67]:
#buscamos elementos verticales en la última modificación de nuestra imagen "eroded"
vertical_kernel = cv.getStructuringElement(cv.MORPH_RECT, (1, 100))
detected_lines_vertical = cv.morphologyEx(eroded, cv.MORPH_OPEN, vertical_kernel, iterations=1)
cnts_vertical, _ = cv.findContours(detected_lines_vertical, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

#creamos la máscara y sustituimos
mask_vertical = np.zeros_like(eroded)
for cnt_vertical in cnts_vertical:
    x, y, w, h = cv.boundingRect(cnt_vertical)
    cv.rectangle(mask_vertical, (x, y), (x + w, y + h), 255, -1)

#invertimos la máscara
mask_inverted_vertical = cv.bitwise_not(mask_vertical)

#la aplicamos a nuestra imagen para que sustituya esos píxeles
clean_img = cv.bitwise_and(eroded, eroded, mask=mask_inverted_vertical)

In [68]:
#vamos a ver cómo se ve ahora nuestra imagen sin línea vertical
cv.imshow('imagen sin líneas verticales', clean_img)
cv.imwrite('./img/imagen_limpia_2.jpg', clean_img)
cv.waitKey(0)
cv.destroyAllWindows()

![Imagen limpia hasta ahora](\img\imagen_limpia_2.jpg)

Ya tenemos nuestra imagen con objetos a detectar aislados del fondo. Ahora vamos a poner en marcha nuestro algoritmo que detectará en toda la lista de imágenes los contornos y los añadirá a una tabla para entrenar

**ENCAPSULAR**

Encapsulamos en una única función todo el proceso anterior para pasar imagen a imagen por esta transformación:

In [69]:
def cleaning_img(img): 
    rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    blank = np.zeros(img.shape[:2], dtype='uint8')
    b,g,r = cv.split(img)
    blur = cv.GaussianBlur(b, (3,3), cv.BORDER_DEFAULT)
    ret, thresh = cv.threshold(blur, 225, 255, 1, cv.THRESH_BINARY)
    thresh = 255 - thresh
    canny = cv.Canny(thresh, 0, 25)
    dilated = cv.dilate(canny, (15,15), iterations=2)
    eroded = cv.erode(dilated, (7,7), iterations=1)
    #verticales
    vertical_kernel = cv.getStructuringElement(cv.MORPH_RECT, (1, 100))
    detected_lines_vertical = cv.morphologyEx(eroded, cv.MORPH_OPEN, vertical_kernel, iterations=1)
    cnts_vertical, _ = cv.findContours(detected_lines_vertical, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    num_clips = len(cnts_vertical)
    mask_vertical = np.zeros_like(eroded)
    for cnt_vertical in cnts_vertical:
        x, y, w, h = cv.boundingRect(cnt_vertical)
        cv.rectangle(mask_vertical, (x, y), (x + w, y + h), 255, -1)
    mask_inverted_vertical = cv.bitwise_not(mask_vertical)
    clean_img = cv.bitwise_and(eroded, eroded, mask=mask_inverted_vertical)
    
    return clean_img, num_clips
    

Encapsulamos ahora en una función el proceso de **lectura de imágenes** además de **procesamiento**.

In [70]:
def read_data(path):
    X = []
    num_clips_list = []
    for img in os.listdir(path):
        image = cv.imread(os.path.join(path, img))

        if image is not None:
            img_masked, num_clips = cleaning_img(image)

            if img_masked is not None:
                smallimage = cv.resize(img_masked, (96, 96))
                smallimage = smallimage / 255.0  # Normalizar
                X.append(smallimage)  # X: images
                num_clips_list.append(int(num_clips))  # número de clips

    return np.array(X), np.array(num_clips_list)


**CARGAMOS IMÁGENES DE TRAIN Y TEST PARA LA X**

In [71]:
X_train,names_train = read_data(TRAIN_PATH)

In [72]:
X_train.shape

(15000, 96, 96)

In [73]:
X_test,names_test = read_data(TEST_PATH)

In [74]:
X_test.shape

(3000, 96, 96)

In [75]:
X_sol,names_sol = read_data(path_sol)

In [76]:
X_sol.shape

(2000, 96, 96)

**CARGAMOS DF DE TRAIN Y TEST PARA LA Y**

In [77]:
df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)
df_sol = pd.read_csv(sol_csv_path)

In [78]:
print(df_train.shape)
print(df_test.shape)
print(df_sol.shape)

(15000, 2)
(5000, 1)
(5000, 2)


In [79]:
y_train = df_train['clip_count']

In [90]:
y_test = df_sol['clip_count']

In [81]:
y_train.dtype

dtype('int64')

In [91]:
#pasamos a float
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

Comprobamos shape de cada una

In [83]:
X_train.shape

(15000, 96, 96)

In [84]:
y_train.shape

(15000,)

In [85]:
X_test.shape

(3000, 96, 96)

In [86]:
X_sol.shape

(2000, 96, 96)

In [92]:
X_test = np.concatenate([X_test, X_sol])

In [98]:
names_test = np.concatenate([names_test, names_sol])

In [94]:
X_test.shape

(5000, 96, 96)

In [93]:
y_test.shape

(5000,)

---

**APLICACIÓN DE MODELO**

In [95]:
#defino un modelo de cnn sencillo para empezar
model = keras.Sequential([
    layers.Flatten(input_shape=(96, 96)), 
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='linear')    #capa de salida con activación linear para la problemática de regresión
])

#compilamos
model.compile(optimizer='adam',
              loss='mean_squared_error', 
              metrics=['mae'])        

In [99]:
#entrenamiento
model.fit(X_train, names_train, epochs=10)

#evaluación
test_loss, test_mae = model.evaluate(X_test, names_test)
print(f'Test Mean Absolute Error: {test_mae}')


Epoch 1/10
469/469 [==============================] - 12s 11ms/step - loss: 3.1401 - mae: 1.4392
Epoch 2/10
469/469 [==============================] - 5s 11ms/step - loss: 1.5199 - mae: 0.9933
Epoch 3/10
469/469 [==============================] - 5s 11ms/step - loss: 0.9051 - mae: 0.7734
Epoch 4/10
469/469 [==============================] - 6s 12ms/step - loss: 0.4873 - mae: 0.5719
Epoch 5/10
469/469 [==============================] - 5s 11ms/step - loss: 0.2820 - mae: 0.4390
Epoch 6/10
469/469 [==============================] - 5s 11ms/step - loss: 0.1697 - mae: 0.3413
Epoch 7/10
469/469 [==============================] - 5s 11ms/step - loss: 0.1125 - mae: 0.2741
Epoch 8/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0843 - mae: 0.2313
Epoch 9/10
469/469 [==============================] - 6s 12ms/step - loss: 0.0713 - mae: 0.2043
Epoch 10/10
157/157 [==============================] - 7s 2ms/step - loss: 0.6544 - mae: 0.6102
Test Mean Absolute Error: 0.61020994186